In [27]:
import json
import string
import random
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tensorF # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mayuk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mayuk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
#3 Loading the Dataset: intents.json

data_file = open('Data.json').read()
data = json.loads(data_file)

data

{'ourIntents': [{'tag': 'age',
   'patterns': ['how old are you?'],
   'responses': ['I am 22 years old and my birthday is on 17th January']},
  {'tag': 'greeting',
   'patterns': ['hi', 'hello', 'hey', 'heyy'],
   'responses': ['Hi there', 'Hello', 'Hi :)']},
  {'tag': 'goodbye',
   'patterns': ['bye', 'later'],
   'responses': ['Bye', 'take care']},
  {'tag': 'name',
   'patterns': ["what's your name?", 'who are you?'],
   'responses': ['I am Tanjiro']},
  {'tag': 'exit',
   'patterns': ['quit', 'exit'],
   'responses': ['Okay, Good bye!!', 'Bye, Have a nice day ']},
  {'tag': 'greeting',
   'patterns': ['hi there',
    'how are you',
    'is anyone there?',
    'hey',
    'hola',
    'hello',
    'good day',
    'namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['bye',
    'see you later',
    'goodbye',
    'get lost',
    'till next time',
    'bye'],
   'responses

### Processing data

In [29]:
lm = WordNetLemmatizer() #reducing words to their base or dictionary form

ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in data["ourIntents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        documentX.append(pattern)
        documentY.append(intent['tag'])
    if intent["tag"] not in ourClasses:
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation]
newWords = sorted(set(newWords))
ourClasses = sorted(set(ourClasses))

In [30]:
trainingData  = []
outEmpty = [0] * len(ourClasses)

for idx, doc in enumerate(documentX):
    bag0words = []
    text = lm.lemmatize(doc.lower())
    for word in newWords :
        bag0words.append(1) if word in text else bag0words.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1 
    trainingData.append([bag0words, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:,0]))
y = num.array(list(trainingData[:,1]))

#### Defining and training a neural network model 


In [31]:
iShape = (len(x[0]),)
oShape = len(y[0])

Model = Sequential()

Model.add(Dense(256, activation="relu" , input_shape=iShape))

Model.add(Dropout(0.5)) #Dropout is a regularization technique that randomly drops out (sets to zero) some of the inputs to a layer during training to prevent overfitting.

Model.add(Dense(64, activation="relu"))

Model.add(Dropout(0.3))

Model.add(Dense(oShape, activation='softmax'))


md = tensorF.keras.optimizers.Adam(learning_rate= 0.01)

Model.compile(optimizer=md, loss='categorical_crossentropy', metrics=['accuracy'])

print(Model.summary())


C:\Users\mayuk\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 256)                 │          21,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 20)                  │           1,300 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,252 (153.33 KB)

 Trainable params: 39,252 (153.33 KB)

 Non-trainable params: 0 (0.00 B)

None


#### Building useful features

In [33]:
# Train the model and save the history
history = Model.fit(x, y, epochs=200, verbose=1)

# Get the list of accuracy values from the history
accuracy_values = history.history['accuracy']

# Calculate the average accuracy
average_accuracy = sum(accuracy_values) / len(accuracy_values)

print(f"accuracy: {average_accuracy:.4f}")


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9892 - loss: 0.0236 
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9814 - loss: 0.0272 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9814 - loss: 0.0276 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9892 - loss: 0.0299 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9892 - loss: 0.0304 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0229 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9932 - loss: 0.0305 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9814 - loss: 0.0326 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9892 - loss: 0.0227 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9814 - loss: 0.0291 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9814 - loss: 0.0184 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9707 - los

In [34]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = Model.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["ourIntents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [35]:
pip install googletrans==4.0.0-rc1


Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install googletrans==4.0.0-rc1 gtts playsound

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 349.5 kB/s eta 0:00:04
   ----------- ---------------------------- 0.5/1.8 MB 349.5 kB/s eta 0:00:04
   ----------- ---------------------------- 0.5/1.8 MB 349.5 kB/s eta 0:00:04
   ----------------- ---------------------- 0.8/1.8 MB 335.4 kB/s eta 0:00:04
   ----------------- ---------------------- 0.8/1.8 MB 335.4 kB/s eta 0:00:04
   ----------------- ---------------------- 0.8/1.8 MB 335

In [37]:
pip install pygame

In [ ]:
from googletrans import Translator
from gtts import gTTS
import os
import string
from pygame import mixer
import time

# Initialize pygame mixer
mixer.init()

# Initialize the translator
translator = Translator()

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

while True:
    newMessage = input("User: ").lower()
    
    if newMessage.lower() in ["y", "exit", "quit"]:
        print("Goodbye! Have a great day!")
        break
    
    newMessage = remove_punctuation(newMessage)
    translatedMessage = translator.translate(newMessage, src='auto', dest='en').text.lower()
    
    intents = Pclass(translatedMessage, newWords, ourClasses)
    ourResult = getRes(intents, data)
    
    user_lang = translator.detect(newMessage).lang
    translatedResponse = translator.translate(ourResult, src='en', dest=user_lang).text
    
    print("Bot:", translatedResponse)
    
    # Text-to-Speech Conversion
    try:
        tts = gTTS(
            text=translatedResponse,
            lang=user_lang,
            slow=False,
            lang_check=False
        )
        tts.save("bot_response1.mp3")
    
    # Load and play the audio file
        mixer.music.load("bot_response1.mp3")
        mixer.music.play()
    
    # Wait for audio to finish playing
        while mixer.music.get_busy():
            time.sleep(0.1)  # Add small sleep intervals
    
    # Stop and unload the music
        mixer.music.stop()
        mixer.music.unload()
    
    # Cleanup with retry logic
        for _ in range(3):  # Try up to 3 times
            try:
                os.remove("bot_response1.mp3")
                break
            except PermissionError:
                time.sleep(0.1)  # Wait 100ms before retrying
    except Exception as e:
        print(f"Voice assistance error: {str(e)}")
        print("Here's the text response instead:", translatedResponse)

User:  Hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: Hello


User:  what are you?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Bot: I am Tanjiro, a Personal VLSI-Based Chatbot


User:  তুমি কি?


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bot: VLSI (Very Large Scale Integration) is the process of integrating millions (or even billions) of transistors into a single chip to create complex electronic circuits. It is a fundamental concept in modern electronics and semiconductor design.


User:  vlsi কি?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: ভিএলএসআই (খুব বড় আকারের সংহতকরণ) হ'ল জটিল বৈদ্যুতিন সার্কিট তৈরি করতে কয়েক মিলিয়ন (বা এমনকি বিলিয়ন) ট্রানজিস্টরকে একক চিপে সংহত করার প্রক্রিয়া।এটি আধুনিক ইলেকট্রনিক্স এবং সেমিকন্ডাক্টর ডিজাইনের একটি মৌলিক ধারণা।


User:   vlsi అంటే ఏమిటి?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: VLSI (చాలా పెద్ద ఎత్తున ఇంటిగ్రేషన్) అనేది సంక్లిష్ట ఎలక్ట్రానిక్ సర్క్యూట్లను సృష్టించడానికి మిలియన్ల (లేదా బిలియన్లు) ట్రాన్సిస్టర్‌లను ఒకే చిప్‌లోకి అనుసంధానించే ప్రక్రియ.ఇది ఆధునిక ఎలక్ట్రానిక్స్ మరియు సెమీకండక్టర్ డిజైన్‌లో ప్రాథమిక భావన.
